[Reference](https://python.plainenglish.io/build-a-web-scraping-python-project-from-start-to-finish-69039654d2de)

# 1. Understanding our source
To understand how to extract this information we can inspect the element we need to retrieve <br>
ctrl+shift+c and then click on it

# 2. Scraping the data


In [7]:
pip install requests-html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import requests_html

url = 'https://www.cnet.com/tech/computing/{page}/'
sess = requests_html.HTMLSession()

class Article:
    
    def __init__(self, title, url):
        self.title = title
        self.url = url
        self.author = None
        self.short_description = None
        self.date = None
        self.text = None
        
    def get_full_article(self):
        """Get the full article from the url
        """        
        art_sess = requests_html.HTMLSession()
        art_res = art_sess.get(self.url)
        try:
            self.date = art_res.html.find('time',first=True).attrs['datetime']
            full_text = []
            for el in art_res.html.find('.article-main-body',first=True).find('p'):
                full_text.append(el.text)
            self.text = '\n'.join(full_text)
        except KeyError as e:
            print(f'KeyError: {e}')
            
    def __str__(self):
        return "{}, {}".format(self.title, self.url)

# DATA SCRAPING
articles = []
for page in range(1,2):
    res = sess.get(url.format(page=page))
    html = res.html
    for html_article in html.find('.item'):
        title = html_article.find('h3', first=True).text
        print('Scraping article:', title)
        url = [el for el in list(html_article.absolute_links) if 'profiles' not in el][0]
        article = Article(title, url)
        article.short_description = html_article.find('p', first=True).text
        article.author = html_article.find('span', first=True).text.replace('by ','')
        article.get_full_article()
        articles.append(article)
        print('Scraped article:', article.title)
        print('-' * 20)

Scraping article: What's Coming to Apple Arcade in July? Everything to Know
Scraped article: What's Coming to Apple Arcade in July? Everything to Know
--------------------
Scraping article: Best Desktop PC for 2022
Scraped article: Best Desktop PC for 2022
--------------------
Scraping article: Take Control of Windows Updates With These Tips
Scraped article: Take Control of Windows Updates With These Tips
--------------------
Scraping article: Want More From Windows 11? Check Out These 9 Handy Features
Scraped article: Want More From Windows 11? Check Out These 9 Handy Features
--------------------
Scraping article: Sony InZone M9 Gaming Monitor Review: A Bright Spot for PC, PS5 Dualists
Scraped article: Sony InZone M9 Gaming Monitor Review: A Bright Spot for PC, PS5 Dualists
--------------------
Scraping article: Mojo's Smart Contact Lenses Begin In-Eye Testing
Scraped article: Mojo's Smart Contact Lenses Begin In-Eye Testing
--------------------
Scraping article: Best Gaming PC for 2

# 3. Data Storage


In [9]:
import sqlite3

# DATA STORAGE
conn = sqlite3.connect('cnet_articles.db')
conn.execute('''CREATE TABLE IF NOT EXISTS articles ( 
             artTitle TEXT PRIMARY KEY,
             artUrl TEXT,
             artAuthor TEXT,
             artShortDesc TEXT,
             artDate TEXT,
             artText TEXT
             )''')
conn.commit()
for article in articles:
    conn.execute('''INSERT OR IGNORE INTO articles VALUES (?,?,?,?,?,?)''',
                 (article.title, article.url, article.author, article.short_description, article.date, article.text))
conn.commit()

conn.execute('''SELECT * FROM articles''').fetchall()

[('Best Desktop PC for 2022',
  'https://www.cnet.com/tech/computing/best-desktop-pc/',
  'John Falcone',
  'We choose the best options for tower PCs, all-in-ones and desktop Macs you can buy right now.',
  '2022-06-29T04:00:23-0700',
  'While it\'s true that only one out of every five computers sold is a desktop, we think it\'s time for people to give more consideration to desktop computers. Laptops and tablets\xa0sure are portable and convenient, but when you\'re spending nearly your whole day on the computer it can be better to have a big-screen monitor -- or even a multiple-monitor setup.\nThe best feature of desktop PCs is the durability and longevity they provide. Not only are desktops built more solidly, but not moving around much contributes to far less wear and tear than your conventional laptop will see. And another of the best desktop PC features is that you can get a decent bit more power and expandability than you could from a laptop, along with a powerful processor and a 

In [10]:
conn.execute('''SELECT artTitle, artAuthor FROM articles''').fetchall()

[('Best Desktop PC for 2022', 'John Falcone'),
 ('Take Control of Windows Updates With These Tips', 'Attila Tomaschek'),
 ('Want More From Windows 11? Check Out These 9 Handy Features',
  'Alison DeNisco Rayome'),
 ('Sony InZone M9 Gaming Monitor Review: A Bright Spot for PC, PS5 Dualists',
  'Lori Grunin'),
 ("Mojo's Smart Contact Lenses Begin In-Eye Testing", 'Scott Stein'),
 ('Best Gaming PC for 2022', 'Lori Grunin'),
 ('Best Prime Day Laptop Deals: Save $200 On a Samsung Galaxy Book 2 360, $250 On a Dell Inspiron 14 2-in-1',
  'Joshua Goldman'),
 ('Insta360, Leica Made A Modular 360 Camera With 1-Inch Sensors for Immersive 6K Video',
  'Joshua Goldman'),
 ('MacBook Settings to Change as Soon as You Turn It On', 'Matt Elliott'),
 ("Take Your Best Ever Vacation Photos: Essential Gear You'll Need",
  'Andrew Lanxon'),
 ('Best Gifts for Serious Photographers in 2022', 'Andrew Lanxon'),
 ('How to Take a Screenshot on Your Mac: 4 Ways to Capture Your Screen',
  'Matt Elliott'),
 ('Best 3

# 4. Script scheduling


In [11]:
# pip install git+https://github.com/Inzaniak/tasKobra@master

In [12]:
# from taskobra import Task

# path_to_script = "INSERT_YOUR_PATH"
# username = "INSERT_YOUR_USERNAME"

# task = Task("cnet_scraper", "scrapers")
# task.set_schedule("-Daily -At 03:00:00")
# task.create_task(path_to_script, username)
# task.run_task()